In [6]:
import datetime
import pandas as pd
import random
import numpy as np
import seaborn as sns

In [7]:
rng = np.random.default_rng(123)

start_year = 1970
end_year = 2020
portfolios = ["First", "Second", "Third", "Fourth", "Fifth"]
portfolios_list = []
dates_list = []


for portfolio in portfolios:
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            date = datetime.date(year, month, 1)
            dates_list.append(date)
            portfolios_list.append(portfolio)

random_returns = rng.uniform(low=(-0.1 / 12), high=(0.3 / 12), size=len(dates_list))


fake_returns = pd.DataFrame(
    {"Portfolio": portfolios_list, "date": dates_list, "ret": random_returns}
)


In [8]:
fake_returns

,Portfolio,date,ret
0,First,1970-01-01,0.014412
1,First,1970-02-01,-0.006539
2,First,1970-03-01,-0.000988
3,First,1970-04-01,-0.002188
4,First,1970-05-01,-0.002470
...,...,...,...
3055,Fifth,2020-08-01,0.015090
3056,Fifth,2020-09-01,-0.002152
3057,Fifth,2020-10-01,0.018363
3058,Fifth,2020-11-01,0.012217


In [ ]:
# Some pseudo code

# Input variables (these require input from the user, MANDATORY)

submitted_income = 85000
income_growth = 0.035

monthly_income = submitted_income/12
monthly_growth = income_growth/12


# I think I just want these to be static
save_rate_monthly = 0.10/12

start_savings = 25
end_savings = 65
death_date = 85

month_start_savings = 


In [12]:
total_monthly_incomes = []

for t in fake_returns.index:
    


3059